In [ ]:
!pip install chromadb

In [ ]:
import json
from glob import glob
import os

def load_all_books(folder_path):
    data = []
    for file_path in glob(os.path.join(folder_path, "*.json")):
        with open(file_path, 'r') as f:
            content = json.load(f)
            title = content['book_metadata']['title']
            for chapter in content.get('chapters', []):
                chunk = {
                    "title": title,
                    "chapter": chapter.get("title", ""),
                    "summary": chapter.get("summary", ""),
                    "text": chapter.get("text", "")[:3000],  # truncation for indexing
                    "qa": chapter.get("educational_qa_pairs", [])
                }
                data.append(chunk)
    return data

books_data = load_all_books("/content/sample_data/Data")

In [ ]:
len(books_data)

170

In [ ]:
from chromadb import HttpClient
from chromadb.config import Settings

chroma_client = HttpClient(host="34.126.200.250", port=8000)

collection = chroma_client.get_or_create_collection(name="orwell_books")

for i, item in enumerate(books_data):
    collection.add(
        documents=[item["text"]],
        metadatas=[{"title": item["title"], "chapter": item["chapter"]}],
        ids=[f"doc_{i}"]
    )

/root/.cache/chroma/onnx_models/all-MiniLM-L6-v2/onnx.tar.gz: 100%|██████████| 79.3M/79.3M [00:02<00:00, 39.3MiB/s]
